In [1]:
from nltk.tokenize import word_tokenize
from pyspark.mllib.regression import LabeledPoint
import nltk

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1571111030205_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import time
start_time = time.time()
data=sc.textFile("s3://hpcproj//datasetfinal.csv")
head=data.first()

pos = sc.textFile("s3://hpcproj//pos1.txt")
neg = sc.textFile("s3://hpcproj//neg1.txt")
pos_sp = pos.flatMap(lambda line: line.split("\n")).collect()
neg_sp = neg.flatMap(lambda line: line.split("\n")).collect()
all_words = []
documents = []
allowed = ["J", "R", "V", "N"]
for p in pos_sp:
    documents.append({"text": p , "label": 1})

for p in neg_sp:
    documents.append({"text": p , "label": 0})

def wc(data):
    words = word_tokenize(data)
    tag = nltk.pos_tag(words)
    for w in tag:
        if w[1][0] in allowed:
            all_words.append(w[0].lower())

    return all_words


raw_data = sc.parallelize(documents, numSlices=100)
raw_tokenized = raw_data.map(lambda dic : {"text": wc(dic["text"]) , "label" : dic["label"]})



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import nltk

from pyspark import SparkConf, SparkContext

from nltk.tokenize import word_tokenize

from pyspark.mllib.feature import HashingTF, IDF

from pyspark.mllib.regression import LabeledPoint

htf = HashingTF(50000)
raw_hashed = raw_tokenized.map(lambda dic : LabeledPoint(dic["label"], htf.transform(dic["text"])))
raw_hashed.persist()
trained_hashed, test_hashed = raw_hashed.randomSplit([0.7, 0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def s(row):
    split_row=row.split(",")
    return (split_row[0],split_row[1].split(" "))

data=data.filter(lambda r: r!=head)
dataset=data.map(lambda r: s(r))
#convert rdd into sql dataframe to remove stop words, make ngram model and turn review into a feature
dataframe=spark.createDataFrame(dataset, ["sentiment", "tweet"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
dataframe.show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+
|sentiment|               tweet|
+---------+--------------------+
|        0|[is, so, sad, for...|
|        0|[i, missed, the, ...|
|        1|[omg, its, alread...|
|        0|[omgaga, im, sooo...|
|        0|[i, think, mi, bf...|
|        0|[or, i, just, wor...|
|        1|[juuuuuuuuuuuuuuu...|
|        0|[sunny, again, wo...|
|        1|[handed, in, my, ...|
|        1|[hmmmm, i, wonder...|
+---------+--------------------+
only showing top 10 rows

In [6]:
from pyspark.ml.feature import StopWordsRemover
remover=StopWordsRemover(inputCol="tweet", outputCol="filtered")
filtered_df=remover.transform(dataframe)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
filtered_df.show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------------+
|sentiment|               tweet|            filtered|
+---------+--------------------+--------------------+
|        0|[is, so, sad, for...|  [sad, apl, friend]|
|        0|[i, missed, the, ...|[missed, new, moo...|
|        1|[omg, its, alread...|   [omg, already, o]|
|        0|[omgaga, im, sooo...|[omgaga, im, sooo...|
|        0|[i, think, mi, bf...|[think, mi, bf, c...|
|        0|[or, i, just, wor...|       [worry, much]|
|        1|[juuuuuuuuuuuuuuu...|[juuuuuuuuuuuuuuu...|
|        0|[sunny, again, wo...|[sunny, work, tom...|
|        1|[handed, in, my, ...|[handed, uniform,...|
|        1|[hmmmm, i, wonder...|[hmmmm, wonder, n...|
+---------+--------------------+--------------------+
only showing top 10 rows

In [8]:
#now make 2-gram model

from pyspark.ml.feature import NGram
ngram=NGram(n=2, inputCol="filtered", outputCol="2gram")
gram_df=ngram.transform(filtered_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
gram_df.show(n=10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------------+--------------------+
|sentiment|               tweet|            filtered|               2gram|
+---------+--------------------+--------------------+--------------------+
|        0|[is, so, sad, for...|  [sad, apl, friend]|[sad apl, apl fri...|
|        0|[i, missed, the, ...|[missed, new, moo...|[missed new, new ...|
|        1|[omg, its, alread...|   [omg, already, o]|[omg already, alr...|
|        0|[omgaga, im, sooo...|[omgaga, im, sooo...|[omgaga im, im so...|
|        0|[i, think, mi, bf...|[think, mi, bf, c...|[think mi, mi bf,...|
|        0|[or, i, just, wor...|       [worry, much]|        [worry much]|
|        1|[juuuuuuuuuuuuuuu...|[juuuuuuuuuuuuuuu...|[juuuuuuuuuuuuuuu...|
|        0|[sunny, again, wo...|[sunny, work, tom...|[sunny work, work...|
|        1|[handed, in, my, ...|[handed, uniform,...|[handed uniform, ...|
|        1|[hmmmm, i, wonder...|[hmmmm, wonder, n...|[hmmmm wonder, wo...|
+---------+--------------

In [10]:
#now make term frequency vectors out of data frame to feed machine
from pyspark.ml.feature import HashingTF,IDF
hashingtf=HashingTF(inputCol="2gram", outputCol="tf", numFeatures=20000)
tf_df=hashingtf.transform(gram_df)
#tf-idf
idf=IDF(inputCol="tf", outputCol="idftf")
idfModel=idf.fit(tf_df)
idf_df=idfModel.transform(tf_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
idf_df.show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|sentiment|               tweet|            filtered|               2gram|                  tf|               idftf|
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        0|[is, so, sad, for...|  [sad, apl, friend]|[sad apl, apl fri...|(20000,[9509,1977...|(20000,[9509,1977...|
|        0|[i, missed, the, ...|[missed, new, moo...|[missed new, new ...|(20000,[1358,2474...|(20000,[1358,2474...|
|        1|[omg, its, alread...|   [omg, already, o]|[omg already, alr...|(20000,[11657,197...|(20000,[11657,197...|
|        0|[omgaga, im, sooo...|[omgaga, im, sooo...|[omgaga im, im so...|(20000,[1251,2920...|(20000,[1251,2920...|
|        0|[i, think, mi, bf...|[think, mi, bf, c...|[think mi, mi bf,...|(20000,[3128,4257...|(20000,[3128,4257...|
|        0|[or, i, just, wor...|       [worry, much]|        [wo

In [12]:
#convert dataframe t rdd, to make a LabeledPoint tuple(label, feature, vector) for machine
tf_rdd=tf_df.rdd

from pyspark.mllib.linalg import  Vectors as MLLibVectors
#we also need to convert ml.sparsevector mllib.sparse vector, because naive bayes only accepts mllib.sparsevector type
train_dataset=tf_rdd.map(lambda x: LabeledPoint(float(x.sentiment), MLLibVectors.fromML(x.tf)))
#split dataset into train, test
train, test=train_dataset.randomSplit([0.9, 0.1], seed=11)
print(train.first())
print(test.first())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.0,(20000,[9509,19771],[1.0,1.0]))
(0.0,(20000,[1429,2631,3344,3703,5070,5522,9209,17496],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]))

In [13]:
#Naive Bayes Model
#create Model
#now train and save the model

from pyspark.mllib.classification import NaiveBayes
import shutil
#training
print("************************TRAINIG*******************************")
model=NaiveBayes.train(train, 1.0)
print("*****************************TRAINING COMPLETE************************************")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

************************TRAINIG*******************************
*****************************TRAINING COMPLETE************************************

In [14]:
#saving the model

output_dir = 's3://hpcproj//NaiveBayesModel_Tweet3'
shutil.rmtree(output_dir, ignore_errors=True)
model.save(sc,output_dir)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o297.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory s3://hpcproj/NaiveBayesModel_Tweet3/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:36

In [15]:
#testing on test data
print("************************TESTING***********************************")
predictionAndLabel=test.map(lambda x: (x.label, model.predict(x.features)))
accuracy=1.0*predictionAndLabel.filter(lambda x: x[0]==x[1]).count()/test.count()
print("Model Accuracy is ", accuracy)
print("*****************TESTING COMPLETED*****************************")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

************************TESTING***********************************
Model Accuracy is  0.6577124948799283
*****************TESTING COMPLETED*****************************

In [16]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#Loogistic Regression Model
train.cache()
# Run training algorithm to build the model
model1 = LogisticRegressionWithLBFGS.train(train)

#saving the model

output_dir = 's3://hpcproj//LRmodel'
shutil.rmtree(output_dir, ignore_errors=True)
model1.save(sc,output_dir)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o393.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory s3://hpcproj/LRmodel/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apac

In [18]:
#testing
labelsAndPreds = test.map(lambda p: (p.label, model1.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(test.count())
print("Accuracy = " + str(1-trainErr))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.6614322864572915

In [19]:
#Binary Classification
# Compute raw scores on the test set
predictionAndLabels = test.map(lambda lp: (float(model.predict(lp.features)), lp.label))

# Instantiate metrics object
metrics = BinaryClassificationMetrics(predictionAndLabels)

#Precision-Recall curves summarize the trade-off between the true 
#positive rate and the positive predictive value for a predictive model using different probability thresholds.
# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

#ROC Curves summarize the trade-off between the true positive rate 
#and false positive rate for a predictive model using different probability thresholds.
# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Area under PR = 0.5203759771790274
Area under ROC = 0.5951497503617456

In [20]:
#Decision Tree Classification
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
model2 = DecisionTree.trainClassifier(trained_hashed, numClasses=2, categoricalFeaturesInfo={},impurity='gini', maxDepth=5, maxBins=32)

#saving the model

output_dir = 's3://hpcproj//DecisionTreeC'
shutil.rmtree(output_dir, ignore_errors=True)
model2.save(sc,output_dir)

print('Learned classification tree model:')
print(model2.toDebugString())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o492.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory s3://hpcproj/DecisionTreeC/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at or

In [22]:
# Evaluate model on test instances and compute test error
predictions = model2.predict(test_hashed.map(lambda x: x.features))
labelsAndPredictions = test_hashed.map(lambda lp: lp.label).zip(predictions)
testAcc = labelsAndPredictions.filter(lambda lp: lp[0] == lp[1]).count() / float(test_hashed.count())
print('Accuracy = ' + str(testAcc))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.9249915909855365

In [23]:
# Decision Tress Regressor
model3 = DecisionTree.trainRegressor(trained_hashed, categoricalFeaturesInfo={},impurity='variance', maxDepth=5, maxBins=32)

#saving the model

output_dir = 's3://hpcproj//DecisionTreeR'
shutil.rmtree(output_dir, ignore_errors=True)
model3.save(sc,output_dir)

print('Learned regression tree model:')
print(model3.toDebugString())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o603.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory s3://hpcproj/DecisionTreeR/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at or

In [24]:
# Evaluate model on test instances and compute test error
predictions1 = model3.predict(test_hashed.map(lambda x: x.features))
labelsAndPredictions1 = test_hashed.map(lambda lp: lp.label).zip(predictions1)
testMSE = labelsAndPredictions1.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() /float(test_hashed.count())
print('Test Mean Squared Error = ' + str(testMSE))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Mean Squared Error = 0.04477699404218303

In [25]:
# RAndom Forest
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
model4 = RandomForest.trainClassifier(trained_hashed, numClasses=2, categoricalFeaturesInfo={},numTrees=3, featureSubsetStrategy="auto",
                                         impurity='gini', maxDepth=4, maxBins=32)

#saving the model
output_dir = 's3://hpcproj//RandomForest'
shutil.rmtree(output_dir, ignore_errors=True)
model4.save(sc,output_dir)

print('Learned classification forest model:')
print(model4.toDebugString())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o714.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory s3://hpcproj/RandomForest/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org

In [27]:
# Evaluate model on test instances and compute test error
predictions2 = model4.predict(test_hashed.map(lambda x: x.features))
labelsAndPredictions2 = test_hashed.map(lambda lp: lp.label).zip(predictions2)
testAcc1 = labelsAndPredictions2.filter(lambda lp: lp[0] == lp[1]).count() / float(test_hashed.count())
print('Accuracy = ' + str(testAcc1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.7628657921291625

In [28]:
#SVM
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
model5 = SVMWithSGD.train(train, iterations=100)
    
#saving the model
output_dir = 's3://hpcproj//SVM'
shutil.rmtree(output_dir, ignore_errors=True)
model5.save(sc,output_dir)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o839.save.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory s3://hpcproj/SVM/metadata already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.s

In [30]:
# Evaluating the model on training data
labelsAndPreds1 = test.map(lambda p: (p.label, model5.predict(p.features)))
trainErr3 = labelsAndPreds1.filter(lambda lp: lp[0] != lp[1]).count() / float(test.count())
print("Accuracy = " + str(1-trainErr3))
print("--- %s seconds ---" % (time.time() - start_time))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.6373512797797655
--- 1523.3008439540863 seconds ---